In [1]:
from preprocessing import contents_clean, contents_separation, distribution
from preprocessing.morphanal import analyze_corpus
import pandas as pd
from tqdm import tqdm

In [4]:
df = pd.read_csv("a.csv")

In [5]:
df['clean'] = df['본문'].apply(lambda x : contents_clean(x))

In [6]:
df['sep'] = df['clean'].apply(lambda x : contents_separation(x))

In [7]:
df2 = pd.DataFrame(columns = ['str', 'idx'])
for idx, lst in tqdm(enumerate(df['sep'].tolist())):
    split_df= distribution(idx,lst)
    df2 = pd.concat([df2,split_df])
else:
    df2.dropna(axis=0, inplace=True)
    df2.reset_index(drop=True, inplace=True)
df2

50it [00:00, 1193.60it/s]


,str,idx
0,7월 개봉 예정 영화 신작 영화추천 신작 영화 추천 극장 개봉 예정 영화 7월 리부...,0
1,문제는 쥬라기 월드 탑건 신비한 동물사전 레전드 영화의 리부트가 연일 관객들의 기대...,0
2,어린 시절의 향수를 진하게 불러주는 오마주는 반갑지만 그때 느꼈던 감동과 재미는 다...,0
3,저야 극장에서 보는 어지간한 영화는 재밌게 보고 나오는 편입니다만 그래서 극장 관람...,0
4,그래서 2차 3차 가공해서 영화를 해석하고 추천하는 글을 쓰는 데는 생각이 참 많아...,0
...,...,...
2426,왜냐하면 저 위로 올라가야하는데 아이혼자 클라이밍으로 올라가서 높은 곳을 걸어야하거든요,49
2427,저희 아이는 1시간동안 땀이 뻘뻘 나도록 미끄럼틀을 오르내렸어요,49
2428,저 혹등고래 입속으로 들어가면 피노키오 인형이 있답니다,49
2429,두빛나래와 국립해양생물자원관은 장항에 위치해서 하루 여행 코스로 좋은 것 같아요,49


In [8]:
import pickle
# 데이터프레임을 피클 파일로 저장
with open('b.pkl', 'wb') as file:
    pickle.dump(df2, file)

In [ ]:
# 저장된 피클 파일을 불러오기
with open('b.pkl', 'rb') as file:
    df2 = pickle.load(file)
df2

In [ ]:
noun_df, verb_df, adj_df, josa_df, df3 = analyze_corpus(df2)